In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

/home/octav/gitrepos/tum-thesis


In [2]:
model_type = "molemb"
sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
args = get_args()
outdir_suffix = "dev2"
args.n_epochs = 4000 
args.init_model = "{}-base-longer-softmax5".format(model_type)
# args.init_model = "{}-test1".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
args.batch_size = 50 

args.penalty_gumbel = False 

args.scale_lambdas = True
args.connectivity = False 
args.valency = False 
args.euler_characteristic_penalty = False
args.conn_lambda_start = 1
args.conn_lambda_end = 30
args.conn_lambda_epochs_start = 12000
args.conn_lambda_epochs_end = 14000
args.valency_lambda_start = 10
args.valency_lambda_end = 50
args.valency_lambda_epochs_start = 12000
args.valency_lambda_epochs_end = 14000
args.euler_lambda_start = 1
args.euler_lambda_end = 20 
args.euler_lambda_epochs_start = 12000
args.euler_lambda_epochs_end = 14000
args.tau_start = 1
args.tau_end = 0.001
args.tau_epochs_start = 1
args.tau_epochs_end = 16000


args.conn_penalty_function = "capped_logdet" 

print(args, '\n')

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, batch_size=50, conn_lambda_end=30, conn_lambda_epochs_end=14000, conn_lambda_epochs_start=12000, conn_lambda_start=1, conn_penalty_function='capped_logdet', connectivity=False, connectivity_hard=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=False, euler_lambda_end=20, euler_lambda_epochs_end=14000, euler_lambda_epochs_start=12000, euler_lambda_start=1, ffn_activation='LeakyReLU', init_decoder_model='molemb-base-longer-softmax5_decode', init_model='molemb-base-longer-softmax5', linear_out=False, max_num_atoms=70, model_type='molemb', n_epochs=4000, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=200, n_labels=1, n_layers=5, one_batch_train=True, ot_solver='emd', output_dir='mol_opt/output_dev2/molemb-base-longer-softmax5', pc_hidden=150, penalty_gumbel=False, pred_hidden=150, scale_lambdas=True, sinkhorn

In [3]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, same_number_atoms = True)
train_data_loader = get_loader("molgen/data/chembl50", "train", args.batch_size, same_number_atoms = True)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

Found previous model mol_opt/output_dev2/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_1, epoch 1. Overwriting args.
Epoch: 2


/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


FGW torch.Size([29508, 5]) 7.077011105138808e-05
Penalty params: tau=0.99914 conn_l=1.00000 val_l=10.00000 euler_l=1.00000 epoch=2 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 2, train
 fgw:9.4266620
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:9.4266620
Measure Epoch 2, train
 similarity:0.0024464
 penlog:1.1317933
Metrics Epoch 2, train
 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:65.4782609
 batch_molecular_disconnected_validity:100.0000000
 batch_connected_components:23.0000000
 batch_invalid_valency_nodes:100.0000000
 batch_nodes_0degree:23.0000000
 batch_nodes_7plus_degree:0.0000000
 invalid_euler_toofew:100.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-23.0000000
 batch_node_degree:0.0000000
Logits [7.925220012664795, 6.284742832183838, 86.83892059326172]
Epoch duration: 2.524968385696411
mol_opt/output_dev2/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_2
Epoch

/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


FGW torch.Size([29508, 5]) 0.00036452969652600586
Penalty params: tau=0.99871 conn_l=1.00000 val_l=10.00000 euler_l=1.00000 epoch=3 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 3, train
 fgw:52.9514990
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:52.9514990
Measure Epoch 3, train
 similarity:0.0000000
 penlog:-100.0000000
Metrics Epoch 3, train
 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:11.6521739
 batch_molecular_disconnected_validity:0.0000000
 batch_connected_components:1.6200000
 batch_invalid_valency_nodes:59.0434783
 batch_nodes_0degree:0.6200000
 batch_nodes_7plus_degree:19.5800000
 invalid_euler_toofew:0.0000000
 invalid_euler_toomany:100.0000000
 avg_euler_error:127.0600000
 batch_node_degree:26.4608696
Logits [19.55836296081543, 14.915124893188477, 104.22269439697266]
Epoch duration: 2.141371488571167
mol_opt/output_dev2/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_3


...

Epoch: 459
FGW torch.Size([29508, 5]) 7.741754234302789e-05
Penalty params: tau=0.82022 conn_l=1.00000 val_l=10.00000 euler_l=1.00000 epoch=459 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 459, train
 fgw:0.2088293
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2088293
Measure Epoch 459, train
 similarity:0.0698879
 penlog:-5.5729834
Metrics Epoch 459, train
 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:100.0000000
 batch_molecular_disconnected_validity:94.0000000
 batch_connected_components:11.0400000
 batch_invalid_valency_nodes:12.3478261
 batch_nodes_0degree:2.6800000
 batch_nodes_7plus_degree:0.0000000
 invalid_euler_toofew:96.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-10.9800000
 batch_node_degree:1.6956522
Logits [31.801462173461914, 2.6641530990600586, 56.77625274658203]
Epoch duration: 2.114142656326294
mol_opt/output_dev2/molemb-base-longer-softmax5/model_molemb-base-long

Epoch: 468
FGW torch.Size([29508, 5]) 7.57559246267192e-05
Penalty params: tau=0.81704 conn_l=1.00000 val_l=10.00000 euler_l=1.00000 epoch=468 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 468, train
 fgw:0.2079844
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2079844
Measure Epoch 468, train
 similarity:0.0705543
 penlog:-1.5350692
Metrics Epoch 468, train
 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:100.0000000
 batch_molecular_disconnected_validity:98.0000000
 batch_connected_components:11.0800000
 batch_invalid_valency_nodes:11.9130435
 batch_nodes_0degree:2.7000000
 batch_nodes_7plus_degree:0.0000000
 invalid_euler_toofew:94.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-10.9400000
 batch_node_degree:1.6939130
Logits [33.02677917480469, 2.756649971008301, 57.41078186035156]
Epoch duration: 2.2436015605926514
mol_opt/output_dev2/molemb-base-longer-softmax5/model_molemb-base-longer

Epoch: 477
FGW torch.Size([29508, 5]) 7.435221050400287e-05
Penalty params: tau=0.81387 conn_l=1.00000 val_l=10.00000 euler_l=1.00000 epoch=477 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 477, train
 fgw:0.1951527
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.1951527
Measure Epoch 477, train
 similarity:0.0886130
 penlog:-5.6834139
Metrics Epoch 477, train
 batch_molecular_validity:2.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:100.0000000
 batch_molecular_disconnected_validity:94.0000000
 batch_connected_components:10.9200000
 batch_invalid_valency_nodes:11.7391304
 batch_nodes_0degree:2.6200000
 batch_nodes_7plus_degree:0.0000000
 invalid_euler_toofew:94.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-10.8400000
 batch_node_degree:1.6956522
Logits [33.517555236816406, 2.81040096282959, 58.03749465942383]
Epoch duration: 2.025338649749756
mol_opt/output_dev2/molemb-base-longer-softmax5/model_molemb-base-longer

Epoch: 486
FGW torch.Size([29508, 5]) 7.684392767259851e-05
Penalty params: tau=0.81072 conn_l=1.00000 val_l=10.00000 euler_l=1.00000 epoch=486 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 486, train
 fgw:0.2056659
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2056659
Measure Epoch 486, train
 similarity:0.0948989
 penlog:-7.5551777
Metrics Epoch 486, train
 batch_molecular_validity:2.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:100.0000000
 batch_molecular_disconnected_validity:92.0000000
 batch_connected_components:10.4200000
 batch_invalid_valency_nodes:10.1739130
 batch_nodes_0degree:2.1800000
 batch_nodes_7plus_degree:0.0000000
 invalid_euler_toofew:92.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-10.3200000
 batch_node_degree:1.7460870
Logits [34.51192855834961, 2.881490468978882, 58.70412063598633]
Epoch duration: 2.1274166107177734
mol_opt/output_dev2/molemb-base-longer-softmax5/model_molemb-base-longe

Epoch: 495
FGW torch.Size([29508, 5]) 7.652710337424651e-05
Penalty params: tau=0.80757 conn_l=1.00000 val_l=10.00000 euler_l=1.00000 epoch=495 mode=[0 0 0] conn=False euler=False val=False
Losses Epoch 495, train
 fgw:0.2047103
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.2047103
Measure Epoch 495, train
 similarity:0.0856275
 penlog:-5.6929483
Metrics Epoch 495, train
 batch_molecular_validity:2.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:100.0000000
 batch_molecular_disconnected_validity:94.0000000
 batch_connected_components:10.3200000
 batch_invalid_valency_nodes:9.8260870
 batch_nodes_0degree:2.1400000
 batch_nodes_7plus_degree:0.0000000
 invalid_euler_toofew:96.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-10.2600000
 batch_node_degree:1.7530435
Logits [35.34276580810547, 2.94803786277771, 59.203556060791016]
Epoch duration: 1.9927616119384766
mol_opt/output_dev2/molemb-base-longer-softmax5/model_molemb-base-longer

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

In [6]:
for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break
    
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize(*yhat_logits)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

from mol_opt.ot_utils import FGW 
fgw_loss = FGW(alpha = 0.5)
fgw_loss(*pred_pack, tau = 1)

ArgumentError: Python argument types in
    rdkit.Chem.rdmolops.Kekulize(NoneType)
did not match C++ signature:
    Kekulize(RDKit::ROMol {lvalue} mol, bool clearAromaticFlags=False)

In [30]:
torch.optim.Adam([torch.autograd.Variable(torch.Tensor([0.]))]).param_groups[0]['lr']

0.001

In [14]:
0.9 ** (1/5)

0.9791483623609768

In [28]:
vars(args)

{'cuda': True,
 'output_dir': 'mol_opt/output_dev1/pointwise-test5',
 'tb_logs_dir': 'mol_opt/logs_dev1/pointwise-test5',
 'init_model': 'pointwise-test5',
 'init_decoder_model': 'pointwise-test5_decode',
 'task': 'qed',
 'model_type': 'pointwise',
 'one_batch_train': True,
 'batch_size': 50,
 'pred_hidden': 150,
 'pc_hidden': 100,
 'ffn_activation': 'LeakyReLU',
 'n_epochs': 1000,
 'dim_tangent_space': 40,
 'n_layers': 5,
 'n_hidden': 250,
 'n_ffn_hidden': 100,
 'linear_out': False,
 'dropout_gcn': 0.0,
 'dropout_ffn': 0.0,
 'agg_func': 'sum',
 'batch_norm': False,
 'N_transformer': 6,
 'n_ffn_transformer': 100,
 'n_heads_transformer': 10,
 'dropout_transformer': 0.1,
 'ot_solver': 'emd',
 'sinkhorn_entropy': 0.1,
 'sinkhorn_max_it': 10000,
 'connectivity': False,
 'valency': True,
 'euler_characteristic_penalty': True,
 'annealing_rate': 0.002,
 'connectivity_lambda': 0,
 'valency_lambda': 0,
 'euler_lambda': 1,
 'connectivity_hard': False,
 'valency_hard': False,
 'scale_lambdas': F

In [32]:
from tensorboardX import SummaryWriter
writer = SummaryWriter("/home/octav/gitrepos/tum-thesis/mol_opt/dev7")

In [31]:
tags = [
    "pointwise10-dev8/train_avg_euler_error/mean",
    "pointwise10-dev8/train_avg_euler_error/bot_band",
    "pointwise10-dev8/train_avg_euler_error/top_band"
]
# writer.add_custom_scalars_marginchart(tags)


In [2]:
# writer.add_scalar("plm2/plm2", 0, 1)
# writer.add_scalar("plm2/plm2", 1, 2)
# writer.add_scalar("plm2/plm2", 0, 3)
# writer.add_scalar("plm2/plm2", 1, 4)
tags = ["plm", "plm", "plm"]
# layout = {"plm" : {"plm" : ["Margin", tags]}}
layout = {"plm1" : {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]},
          "plm2": {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]}}
writer.add_custom_scalars(layout)

In [33]:
writer._get_file_writer().add_summary()

TypeError: add_summary() missing 1 required positional argument: 'summary'

In [34]:
writer.logdir

'/home/octav/gitrepos/tum-thesis/mol_opt/dev6'

In [11]:
writer.close()

In [4]:
import tensorboardX
tensorboardX.__version__

'2.1'

In [6]:
from tensorboardX.summary import custom_scalars

In [16]:
writer._get_file_writer().add_summary(custom_scalars(layout), global_step = 3)
writer.flush()

In [34]:
writer.add_scalars('plm', {'plm0' : 0.0, 'plmt' : 0.1, 'plmb' : -0.1}, 3)
writer.flush()

In [ ]:
tau = 1
symbols_nll, charges_nll, bonds_nll = F.gumbel_softmax(tau = tau, dim=1, logits = symbols_logits), F.gumbel_softmax(tau=tau,dim=1,logits=charges_logits), F.gumbel_softmax(tau=tau,dim=1, logits = bonds_logits)

In [ ]:
np.sort(symbols_nll.mean(axis = 0).detach().numpy())

In [ ]:
list(molopt.opt0.parameters())[0]

In [ ]:
molopt.state_dict()

In [ ]:
model_dict = torch.load('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8')

In [ ]:
model_dict["model"]['opt0.weight']

In [ ]:
molopt2 = MolOpt(args)

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt2.load_state_dict(model_dict["model"])

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt3,args3 = load_model('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8', MolOpt, args.device)

In [ ]:
list(molopt3.opt0.parameters())[0]

In [ ]:
fgw_loss(*pred_pack, tau = 1)